In [13]:
# Analyze atomic SO-CI output file from Molpro
# KKI 12/21/2023
import re, sys, glob, subprocess, os

os.environ['OMP_NUM_THREADS'] = '1'  # to avoid memory leak in KMeans()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import KMeans

import chem_subs as chem
import molpro_subs as mpr

pd.set_option('display.max_rows', None)
np.set_printoptions(suppress=True)

### Specify Molpro SO-CI output file

In [14]:
fdir = r'C:\Users\irikura\OneDrive - NIST\Karl\UMemphis'
fname = 'Ni_II-29D17Q-12-21-23.out'
#fname = 'ni+_5D7Q_actz_dk.pro'

In [15]:
fsoc = os.sep.join([fdir, fname])
print(f'Reading MOLPRO file "{fsoc}"')
atom = mpr.stoichiometry(fsoc)
charge = mpr.total_charge(fsoc, verbose=True)
print(f'The atom is {atom} with charge {charge}')
# append charge, to match sheet names in exptl data file
if charge > 0: 
    atom += '+'
elif charge < 0:
    atom += '-'
if abs(charge) > 1:
    atom += f'{abs(charge)}'
PG = mpr.read_compgroup(fsoc)
print(f'The computational point group is {PG}')

Reading MOLPRO file "C:\Users\irikura\OneDrive - NIST\Karl\UMemphis\Ni_II-29D17Q-12-21-23.out"
The atom is Ni+1 with charge 1.0
The computational point group is Ci


In [16]:
SOCI = mpr.fullmatSOCI(fsoc, atom=True)

Computational group = Ci
CASSCF states:
    29 Doublet
    10 Quartet


In [17]:
SOCraw = SOCI.vals.min()
print(f'From lowest level and lowest uncoupled energy, raw theoretical SOCraw = {SOCraw:.3f} cm-1')

From lowest level and lowest uncoupled energy, raw theoretical SOCraw = -610.986 cm-1


In [18]:
fmt = {'Eshift': '{:.1f}', 'Pct': '{:.3f}', 'eV': '{:.3f}', 'degen': '{:.0f}'}
for col in ['J', 'Ecalc', 'E_dif', 'Erel', 'Eshift', 'err', 'Eterm', 'cm-1', 'fitted',
           'wmean', 'wstds', 'uwmean', 'uwstds']:
    fmt[col] =  fmt['Eshift']
for col in ['dif', 'Theory', 'ecm', 'SOC', 'RMSE']:
    fmt[col] = '{:.2f}'

In [19]:
dfterm = SOCI.average_terms(be_close=['Energy', 'Edav', 'Eref', 'dipZ', 'C0'], always=True)
# drop the dipZ column
dfterm.drop(columns=['dipZ'], inplace=True)
print('Averaged terms from MRCI:')
dfso = SOCI.assign_atomic_J(quiet=True)  # create SOCI.dfso
Egl = SOCI.dfso.E.min()  # energy of ground level
dfterm['Erel'] = (dfterm.Edav - Egl) * chem.AU2CM
display(dfterm.style.format(fmt))
print('"ecm"  is relative to the lowest term (cm-1)')
print('"Erel" is relative to the ground level')

Averaged terms from MRCI:


,Term,Edav,idx,ecm,Erel
0,(1)2D,-1519.144080,[1 3 0 2 4],0.00,611.0
1,(1)4F,-1519.110837,[33 29 30 31 32 35 34],7295.90,7906.8
2,(1)2F,-1519.089162,[ 5 7 8 6 11 9 10],12053.00,12664.0
3,(1)4P,-1519.038924,[37 38 36],23078.90,23689.9
4,(2)2D,-1519.035648,[12 13 16 14 15],23798.10,24409.1
5,(1)2P,-1519.017203,[17 19 18],27846.20,28457.2
6,(1)2G,-1518.997487,[21 20 24 23 26 28 25 22 27],32173.40,32784.3


"ecm"  is relative to the lowest term (cm-1)
"Erel" is relative to the ground level


In [20]:
irreps_ci = set(SOCI.dfci.Irrep)
if (PG == 'Ci') and (len(irreps_ci) == 1):
    if 1 in irreps_ci:
        parity = 'even'
    else:
        parity = 'odd'

In [21]:
def term_distrib(term, df):
    # return the weights (including 2J+1) of term in levels
    global SOCI
    itarget = SOCI.dfterm[SOCI.dfterm.Term == term].index[0]
    wt = [twt[itarget] for twt in df.termwt]  # without 2J+1 weighting
    wt = wt * (2*df.J + 1)
    return wt

### Display the distribution of one term among levels

In [22]:
term = '(1)2D'

In [23]:
print(f'Distribution of term "{term}" among levels:')
thrsh = 1.e-6
dfdistrib = dfso.copy()
dfdistrib[term] = term_distrib(term, dfso)
dfdistrib['Pct'] = dfdistrib[term] * 100 / dfdistrib[term].sum()
# add column with Eshift expressed in eV
dfdistrib['eV'] = dfdistrib['Eshift'] / chem.EV2CM
# remove rows with negligible weights
dfdistrib = dfdistrib[dfdistrib[term] > thrsh]
dfdistrib.drop(['termwt', 'Composition'], axis=1, inplace=True)
display(dfdistrib.sort_values(term, ascending=False).style.format(fmt))
print(f'Total weight of {term} = {dfdistrib[term].sum():.3f}')
print('Column "Eshift" is relative to the ground term, in cm-1.  "eV" is the same quantity.')

Distribution of term "(1)2D" among levels:


,Lead,J,Jlbl,Erel,Eshift,E,Nr,(1)2D,Pct,eV
0,(1)2D,2.5,(1)2D_5/2,0.0,-610.9,-1519.146863,"[1, 2, 3, 4, 5, 6]",5.999792,59.998,-0.076
1,(1)2D,1.5,(1)2D_3/2,1526.1,915.1,-1519.139910,"[7, 8, 9, 10]",3.999682,39.997,0.113
5,(1)4F,1.5,(1)4F_3/2,9177.4,8566.4,-1519.105048,"[35, 36, 37, 38]",0.000293,0.003,1.062
4,(1)4F,2.5,(1)4F_5/2,8615.6,8004.7,-1519.107608,"[29, 30, 31, 32, 33, 34]",0.000115,0.001,0.992
7,(1)2F,2.5,(1)2F_5/2,13534.1,12923.2,-1519.085198,"[47, 48, 49, 50, 51, 52]",0.000049,0.000,1.602
12,(2)2D,2.5,(2)2D_5/2,25029.4,24418.4,-1519.032821,"[69, 70, 71, 72, 73, 74]",0.000027,0.000,3.027
8,(1)4P,2.5,(1)4P_5/2,22787.0,22176.0,-1519.043038,"[53, 54, 55, 56, 57, 58]",0.000017,0.000,2.749
9,(1)4P,1.5,(1)4P_3/2,23630.6,23019.6,-1519.039195,"[59, 60, 61, 62]",0.000015,0.000,2.854
13,(1)2P,1.5,(1)2P_3/2,28420.1,27809.1,-1519.017372,"[75, 76, 77, 78]",0.000008,0.000,3.448
11,(2)2D,1.5,(2)2D_3/2,24583.0,23972.0,-1519.034855,"[65, 66, 67, 68]",0.000002,0.000,2.972


Total weight of (1)2D = 10.000
Column "Eshift" is relative to the ground term, in cm-1.  "eV" is the same quantity.


In [24]:
dfso.style.format(fmt)

,Lead,J,Jlbl,Erel,Eshift,Composition,E,Nr,termwt
0,(1)2D,2.5,(1)2D_5/2,0.0,-610.9,{'(1)2D': 1.0},-1519.146863,"[1, 2, 3, 4, 5, 6]",[0.99996532 0.00002167 0.00000623 0.0000064 0.00000039 0. 0. ]
1,(1)2D,1.5,(1)2D_3/2,1526.1,915.1,{'(1)2D': 1.0},-1519.139910,"[7, 8, 9, 10]",[0.99992061 0.00007112 0. 0.0000013 0.00000378 0.00000318 0. ]
2,(1)4F,4.5,(1)4F_9/2,6886.6,6275.7,{'(1)4F': 1.0},-1519.115486,"[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]",[0. 0.99935523 0. 0. 0. 0. 0.00064476]
3,(1)4F,3.5,(1)4F_7/2,7822.7,7211.8,"{'(1)4F': 0.98, '(1)2F': 0.02}",-1519.111220,"[21, 22, 23, 24, 25, 26, 27, 28]",[0. 0.98095287 0.01878595 0. 0. 0. 0.00026118]
4,(1)4F,2.5,(1)4F_5/2,8615.6,8004.7,{'(1)4F': 0.99},-1519.107608,"[29, 30, 31, 32, 33, 34]",[0.0000192 0.989862 0.00863752 0.00000622 0.00147506 0. 0. ]
5,(1)4F,1.5,(1)4F_3/2,9177.4,8566.4,{'(1)4F': 1.0},-1519.105048,"[35, 36, 37, 38]",[0.00007313 0.99545674 0.00000001 0.00000331 0.00445729 0.00000953 0. ]
6,(1)2F,3.5,(1)2F_7/2,12066.3,11455.3,"{'(1)4F': 0.02, '(1)2F': 0.98}",-1519.091886,"[39, 40, 41, 42, 43, 44, 45, 46]",[0. 0.01888886 0.98048785 0. 0. 0. 0.00062329]
7,(1)2F,2.5,(1)2F_5/2,13534.1,12923.2,{'(1)2F': 0.99},-1519.085198,"[47, 48, 49, 50, 51, 52]",[0.00000823 0.00806805 0.98617955 0.0000547 0.00568947 0. 0. ]
8,(1)4P,2.5,(1)4P_5/2,22787.0,22176.0,"{'(1)4P': 0.76, '(2)2D': 0.24}",-1519.043038,"[53, 54, 55, 56, 57, 58]",[0.00000283 0.0006208 0.00170991 0.75893724 0.23872921 0. 0. ]
9,(1)4P,1.5,(1)4P_3/2,23630.6,23019.6,"{'(1)4P': 0.69, '(2)2D': 0.29, '(1)2P': 0.02}",-1519.039195,"[59, 60, 61, 62]",[0.00000376 0.00142893 0. 0.68954721 0.28629282 0.02272728 0. ]
